**This is a python code for generating channle model with inteference from the simulated received power of single antenna propagation**

In [15]:
import numpy as np
import random
import matplotlib.pyplot as plt # import the matplotlib module
from mpl_toolkits.mplot3d import Axes3D # import the 3D plotting toolkit
import math
from scipy import special

filename = 'Tx_1_Power.txt'

#define the range of motion and prediction#
x_min = 5
x_max = 46
y_min = -49
y_max = -1     
t = 2058              #define time steps
K_max = 10                #define the maximum number of interferers
trans_ori = (46, -49)                     #define the origin of transmiter

x = [i for i in range(x_min, x_max+1)]    #define x cordinate range of motion and prediction

y = [i for i in range(y_min, y_max+1)]    #define y cordinate range of motion and prediction

x_ori, y_ori = trans_ori


In [16]:
#preprocess the .txt file and return the data matrix

def preprocess(file_name):

    start_line = 14     # Change this to the desired starting line

# Initialize empty lists for each column
    column1 = []
    column2 = []
    column3 = []
    column4 = []

    with open(file_name, 'r') as file:
    # Read and store lines starting from the specified line
        for _ in range(start_line):
            next(file)  # Skip lines until the desired line
        for line in file:
        #data is separated by whitespace, split it into columns
            data = line.split()
        
        #There are four columns, convert each column to the appropriate data type
            col1_value = float(data[0])
            col2_value = float(data[1])
            col3_value = float(data[2])
            col4_value = float(data[3])
        
        # Append the values to their respective columns
            column1.append(col1_value)
            column2.append(col2_value)
            column3.append(col3_value)
            column4.append(col4_value)


# Convert the lists to NumPy arrays and stack them horizontally to create a matrix, ignore column 3, since the hight is always 1.5m
    data_matrix = np.column_stack((column1, column2, column4))
    return data_matrix
# Now, data_matrix contains the dataset as a NumPy array

In [17]:
#generate Interferer propagation
def generate_interferer(num, dataset):
    #number of interferers
    arrays = []
    K = random.randint(1, num) #K in range (1, num)
    #Define a random position of Transmitter
    #interf_pos = [[0 for j in range(2)] for i in range(K)]
    #T = random.randint(0, t-1) #T in range (0, 2057)
    T_interf = [random.randint(0, t-1) for _ in range(K)] #randomly generate K position of interferers
    propagation_map_interf = np.ones((2058,3))
    #result_interf =  [0,0,0,0]
    #result_interf[:] = T_interf[:]//42
    for i in range(K):
        divided_result_interf = T_interf[i]//42
        modulated_result_interf = T_interf[i]%42
        if divided_result_interf % 2 == 1:

            interf_pos = (x_ori-42+modulated_result_interf, y_ori+divided_result_interf)

        else:
    
            interf_pos = (x_ori-modulated_result_interf, y_ori+divided_result_interf)

        #x_pos_interf[i], y_pos_interf[i] = interf_pos[i,:]
        print(f"Random interferer {i+1} position: {interf_pos}") 


        #propagation map from the random position of Transmitter
    
        for j in range(t):
            propagation_map_interf[j,:] = dataset[j*t+T_interf[i],:]

        #propagation_map_interf_combined.append(propagation_map_interf)   
    
        # Extract x, y, and z values from the propagation_map
        """
        x_mesh = propagation_map_interf[:, 0]
        y_mesh = propagation_map_interf[:, 1]
        z_mesh = propagation_map_interf[:, 2]
        z_mesh_min = np.min(z_mesh)
        z_mesh_max = np.max(z_mesh) 

        # Create a 3D figure
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Plot the 3D surface
        surface = ax.plot_trisurf(x_mesh, y_mesh, z_mesh, cmap='viridis')

        # Set labels and title
        ax.set_xlabel('X-coordinate')
        ax.set_ylabel('Y-coordinate')
        ax.set_zlabel('Propagation Result')
        ax.set_title(f'3D Surface Plot of Interferer {i+1} Propagation Map')

        # Add color bar
        fig.colorbar(surface)
        # Show the plot
        plt.show()
        """

        new_array = propagation_map_interf
        arrays.append(new_array)

    propagation_map_interf_combined = np.stack(arrays)    
    print(propagation_map_interf_combined.shape)

#think about which kind of data to be returned

    return  propagation_map_interf_combined, K 

In [18]:
# Define the propagation area
propagation_area = [(X, Y) for X in range(5, 47) for Y in range(-1, -50, -1)]

# Define the starting point and initialize the route
start_point = (5, -1)
route = [start_point]

# Define a function to get the adjacent points
def get_adjacent_points(point):
    x, y = point
    return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

# Iterate through the propagation area and append each point to the route
while len(route) < len(propagation_area):
    last_point = route[-1]
    adjacent_points = get_adjacent_points(last_point)
    for point in adjacent_points:
        if point in propagation_area and point not in route:
            route.append(point)
            break

# Print the route
for point in route:
    print(point)


(5, -1)
(6, -1)
(7, -1)
(8, -1)
(9, -1)
(10, -1)
(11, -1)
(12, -1)
(13, -1)
(14, -1)
(15, -1)
(16, -1)
(17, -1)
(18, -1)
(19, -1)
(20, -1)
(21, -1)
(22, -1)
(23, -1)
(24, -1)
(25, -1)
(26, -1)
(27, -1)
(28, -1)
(29, -1)
(30, -1)
(31, -1)
(32, -1)
(33, -1)
(34, -1)
(35, -1)
(36, -1)
(37, -1)
(38, -1)
(39, -1)
(40, -1)
(41, -1)
(42, -1)
(43, -1)
(44, -1)
(45, -1)
(46, -1)
(46, -2)
(45, -2)
(44, -2)
(43, -2)
(42, -2)
(41, -2)
(40, -2)
(39, -2)
(38, -2)
(37, -2)
(36, -2)
(35, -2)
(34, -2)
(33, -2)
(32, -2)
(31, -2)
(30, -2)
(29, -2)
(28, -2)
(27, -2)
(26, -2)
(25, -2)
(24, -2)
(23, -2)
(22, -2)
(21, -2)
(20, -2)
(19, -2)
(18, -2)
(17, -2)
(16, -2)
(15, -2)
(14, -2)
(13, -2)
(12, -2)
(11, -2)
(10, -2)
(9, -2)
(8, -2)
(7, -2)
(6, -2)
(5, -2)
(5, -3)
(6, -3)
(7, -3)
(8, -3)
(9, -3)
(10, -3)
(11, -3)
(12, -3)
(13, -3)
(14, -3)
(15, -3)
(16, -3)
(17, -3)
(18, -3)
(19, -3)
(20, -3)
(21, -3)
(22, -3)
(23, -3)
(24, -3)
(25, -3)
(26, -3)
(27, -3)
(28, -3)
(29, -3)
(30, -3)
(31, -3)
(32, -3)
(33, -3

In [19]:
#calculate the SINR

def calculate_sinr(ps, pi, pn):
    #ps_linear = 10 ** (ps / 10)
    #pi_linear = 10 ** (pi / 10)
    #pn_linear = 10 ** (pn / 10)
    
    sinr = 10 * math.log10(ps/(pi + pn))

    return sinr

In [22]:
def main():

    print("Let's creat an SINR map")
    # get propagation map from the data matix of given time point 

    raw_data = preprocess(filename)

    
    #Define a random position of Transmitter
    #trans_pos = (random.randint(x_min, x_max), random.randint(y_min, y_max))
    T = random.randint(0, t-1) #T in range (0, 2057)

    result = T//42


    if result % 2 == 1:

        trans_pos = (x_ori-42+T%42, y_ori+result)

    else:
    
        trans_pos = (x_ori-T%42, y_ori+result)

    x_pos, y_pos = trans_pos
    print(f"Random Transmitter position: ({x_pos}, {y_pos})")


    #propagation map from the random position of Transmitter
    propagation_map = np.ones((2058,3))
    for i in range(0, t):
        propagation_map[i,:] = raw_data[i*t+T,:]

    """
    # Extract x, y, and z values from the propagation_map
    x_mesh = propagation_map[:, 0]
    y_mesh = propagation_map[:, 1]
    z_mesh = propagation_map[:, 2]
    z_mesh_min = np.min(z_mesh)
    z_mesh_max = np.max(z_mesh) 

    # Create a 3D figure
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot the 3D surface
    surface = ax.plot_trisurf(x_mesh, y_mesh, z_mesh, cmap='viridis')

    # Set labels and title
    ax.set_xlabel('X-coordinate')
    ax.set_ylabel('Y-coordinate')
    ax.set_zlabel('Propagation Result')
    ax.set_title('3D Surface Plot of Transmitter Propagation Map')

    # Add color bar
    fig.colorbar(surface)
    # Show the plot
    plt.show()
    """
    num_rounds = 1
    SINR = np.zeros((num_rounds, 2058))
    
    #simulate for 1000 times
    for i in range(0, num_rounds):
        k=0
        for position in route:
            print(f'position is {position}')
            x_ue, y_ue = position
            indices_signal = np.where((propagation_map[:, 0] == x_ue) & (propagation_map[:, 1] == y_ue))
            print(f'indices_signal are {indices_signal}')
            signal_power = propagation_map[indices_signal[0][0], 2]
            signal_power_linear = 10**(signal_power/10)
            print(f'signal_power is {signal_power} dBm')
            print(f'signal_power_linear is {signal_power_linear} mW')

            Interfererence_map, num_interf = generate_interferer(K_max, raw_data)
            # find the indices 
            indices_interf = np.where((Interfererence_map[:, :, 0] == x_ue) & (Interfererence_map[:, :, 1] == y_ue))
            print(f'indices_interf are {indices_interf}')
            interf_power_linear = 0
            for j in range (0, num_interf):
                interf_power_linear += (10**((Interfererence_map[indices_interf[0][j], indices_interf[1][j], 2])/10))
            interf_power_linear = (interf_power_linear/num_interf)/10
            print(f'interf_power is {interf_power_linear} mW')
            # calculate the SINR
            SINR[i,k] = calculate_sinr(signal_power_linear, interf_power_linear, 0)
            print(f'SINR is {SINR[i,k]} dB')
            k+=1
    print(max(SINR))
    print(min(SINR))

if __name__ == "__main__":
    main()    

Let's creat an SINR map
Random Transmitter position: (15, -23)
position is (5, -1)
indices_signal are (array([48], dtype=int64),)
signal_power is -46.1226 dBm
signal_power_linear is 2.4419681767202505e-05 mW
Random interferer 1 position: (36, -33)
Random interferer 2 position: (22, -18)
Random interferer 3 position: (44, -41)
Random interferer 4 position: (11, -31)
Random interferer 5 position: (15, -40)
Random interferer 6 position: (37, -30)
Random interferer 7 position: (12, -30)
Random interferer 8 position: (22, -45)
Random interferer 9 position: (19, -18)
Random interferer 10 position: (23, -35)
(10, 2058, 3)
indices_interf are (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), array([48, 48, 48, 48, 48, 48, 48, 48, 48, 48], dtype=int64))
interf_power is 9.294584926321701e-07 mW
SINR is 14.195099999999998 dB
position is (6, -1)
indices_signal are (array([97], dtype=int64),)
signal_power is -46.0158 dBm
signal_power_linear is 2.5027645796454855e-05 mW
Random interferer 1 positio